In [ ]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#trainデータの読み込み・目的変数と説明変数の分離
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

train_x = train.drop(['label'],axis=1)
train_y = train['label']

#testデータの読み込み
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
#trainデータを4分割し、訓練用：バリデーションを3:1にする
kf = KFold(n_splits=4, shuffle=True, random_state=123)

tr_idx, va_idx = list(kf.split(train_x))[0]#index取得
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [ ]:
#画像pixel値を255で割って正規化・numpy配列に変換
tr_x, va_x = np.array(tr_x/255.0),np.array(va_x/255.0)

#正解ラベルをOne-Hot表現化
tr_y = to_categorical(tr_y,10)#numpy.ndarray
va_y = to_categorical(va_y,10)#numpy.ndarray

print(tr_x.shape,tr_y.shape,va_x.shape,va_y.shape)

In [ ]:
#0~9の各数字の枚数を調べる
count = Counter(train['label'])
display(count)
sns.countplot(x=train['label'])

In [ ]:
#手書きの数字データを描画する
plt.figure(figsize=(12,10))
x,y=10,5
for i in range(20):
    plt.subplot(y,x,i+1)
    plt.imshow(tr_x[i].reshape((28,28)),interpolation='nearest')#28*28にリサイズ
plt.show()

In [ ]:
#NN第一層（隠れ層）の作成
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation, Dropout

# ニューラルネットワーク基盤のクラスSequentialを生成
model = Sequential()

# 入力層を含む最初の層を定義
model.add(
    Input(shape=(tr_x.shape[1],))  # 入力データの形状を指定
)
model.add(
    Dense(
        128,  # ニューロン数
        activation='sigmoid'  # 活性化関数: シグモイド（各クラスの信頼度を出す）
    )
)

In [ ]:
#第2層（出力層）
model.add(
    Dense(
        10,  # ニューロン数
        activation='softmax'  # マルチクラス分類に適したsoftmaxを指定（各クラスの確率を出す）
    )
)

In [ ]:
model.compile(
    #損失関数：クロスエントロピー誤差
    loss='categorical_crossentropy',
    optimizer='adam',
    #学習評価：正解率
    metrics=['accuracy']
)

In [ ]:
#学習を実行
result = model.fit(
    tr_x,
    tr_y,
    epochs=5,#学習回数
    batch_size=100,#ミニバッチサイズ
    validation_data=(va_x,va_y),#検証用データの指定
    verbose=1#学習の進捗表示
)

In [ ]:
#テストデータの予測・提出
result = model.predict(test)

#最大値のインデックスを表示
print([x.argmax() for x in result[:5]])

#予測した数字をnumpy配列に代入
y_test = [x.argmax() for x in result]

In [ ]:
submit_df = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submit_df['Label'] = y_test
#submit_df.head(5)

submit_df.to_csv('submission.csv', index=False)

In [ ]:
#パラメータチューニング

def prepare_data():#データ作成関数
    #trainデータの読み込み・目的変数と説明変数の分離
    train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
    train_x = train.drop(['label'],axis=1)
    train_y = train['label']

    #trainデータを4分割し、訓練用：バリデーションを3:1にする
    kf = KFold(n_splits=4, shuffle=True, random_state=123)
    tr_idx, va_idx = list(kf.split(train_x))[0]#index取得
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    #画像pixel値を255で割って正規化・numpy配列に変換
    tr_x, va_x = np.array(tr_x/255.0),np.array(va_x/255.0)
    #正解ラベルをOne-Hot表現化
    tr_y = to_categorical(tr_y,10)#numpy.ndarray
    va_y = to_categorical(va_y,10)#numpy.ndarray

    return tr_x,tr_y,va_x,va_y

In [ ]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy
import numpy as np

# モデル構築を試行する関数
def objective(trial):
    # データを取得
    tr_x, tr_y, va_x, va_y = prepare_data()

    # モデルの構築
    model = Sequential()
    model.add(Input(shape=(tr_x.shape[1],)))

    # 第一層（ニューロン数の選択）
    n_units_first = trial.suggest_categorical('units_first', [500, 784])
    model.add(Dense(n_units_first, activation='relu'))
    model.add(Dropout(0.4))

    # 隠れ層の数を選択
    n_layers = trial.suggest_categorical('n_layers', ['none', 'one', 'two'])
    if n_layers == 'one':
        n_units_second = trial.suggest_categorical('units_second', [100, 200])
        model.add(Dense(n_units_second, activation='relu'))
    elif n_layers == 'two':
        n_units_second = trial.suggest_categorical('units_second', [100, 200])
        model.add(Dense(n_units_second, activation='relu'))
        n_units_third = trial.suggest_categorical('units_third', [25, 50])
        model.add(Dense(n_units_third, activation='relu'))

    # 出力層
    model.add(Dense(10, activation='softmax'))

    # オプティマイザの選択
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    optimizer = Adam() if optimizer_name == 'adam' else RMSprop()

    # モデルのコンパイル
    model.compile(
        optimizer=optimizer,
        loss=CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # ハイパーパラメータチューニング中の学習
    history = model.fit(
        tr_x, tr_y,
        epochs=10,
        batch_size=100,
        validation_data=(va_x, va_y),
        verbose=0
    )

    # 検証精度を最小化するようにスコアを設定（最大化したいので負符号）
    validation_acc = max(history.history['val_accuracy'])
    return -validation_acc

# Optunaのチューニング実行
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# 最適なパラメータとモデル
print("Best trial:")
print(study.best_trial.params)

In [ ]:
# さらに細かくチューニング
def objective2(trial):
    # データを取得
    tr_x, tr_y, va_x, va_y = prepare_data()

    # 第一層
    n_units_first = 784
    dropout_first = trial.suggest_float('dropout_first', 0.2, 0.4, step=0.05)#ドロップアウト率探索
    activation_first = trial.suggest_categorical('activation_first', ['tanh', 'relu'])#活性化関数探索
    model.add(Dense(n_units_first, activation=activation_first))
    model.add(Dropout(dropout_first))

    #第2層
    n_units_second = 200
    dropout_second = trial.suggest_float('dropout_second', 0.2, 0.4, step=0.05)#ドロップアウト率探索
    activation_second = trial.suggest_categorical('activation_second', ['tanh', 'relu'])#活性化関数探索
    model.add(Dense(n_units_second, activation=activation_second))
    model.add(Dropout(dropout_second))

    # 出力層
    model.add(Dense(10, activation='softmax'))

    # オプティマイザの選択
    optimizer_name = 'rmsprop'
    optimizer = Adam() if optimizer_name == 'adam' else RMSprop()

    # モデルのコンパイル
    model.compile(
        optimizer=optimizer,
        loss=CategoricalCrossentropy(),
        metrics=['accuracy']
    )


    batch_size = trial.suggest_categorical('batch_size', [100, 200])
    # ハイパーパラメータチューニング中の学習
    history = model.fit(
        tr_x, tr_y,
        epochs=10,
        batch_size=batch_size,
        validation_data=(va_x, va_y),
        verbose=0
    )

    # 検証精度を最小化するようにスコアを設定（最大化したいので負符号）
    validation_acc = max(history.history['val_accuracy'])
    return -validation_acc

# Optunaのチューニング実行
study = optuna.create_study(direction='minimize')
study.optimize(objective2, n_trials=100)

# 最適なパラメータとモデル
print("Best trial:")
print(study.best_trial.params)